In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import random

def handle_no_such_element_exception(data_extraction_task):
    try:
        return data_extraction_task()
    except NoSuchElementException as e:
        return None

# Create a Chrome web driver instance
driver = webdriver.Chrome(service=Service())

# Connect to the target page
url = "https://www.booking.com/searchresults.de.html?label=gen173nr-1BCAEoggI46AdIM1gEaDuIAQGYAQe4ARfIAQzYAQHoAQGIAgGoAgO4Ap-wuLoGwAIB0gIkMWVhOGIyYzUtNzkwZC00ODI3LWE5Y2QtNzY3Mjg5YjA0NGI12AIF4AIB&sid=4daa01aeadda31e7f02b8ddc59e9f35f&aid=304142&ss=Berlin&ssne=Berlin&ssne_untouched=Berlin&efdco=1&lang=de&sb=1&src_elem=sb&dest_id=-1746443&dest_type=city&checkin=2025-05-05&checkout=2025-05-10&ltfd=5%3A1%3A6-2025%3A1%3A&group_adults=2&no_rooms=1&group_children=0&order=distance_from_search"
driver.get(url)

# Handle the sign-in alert
try:
    close_button = WebDriverWait(driver, 12).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[role=\"dialog\"] button[aria-label=\"Dismiss sign-in info.\"]"))
    )
    close_button.click()
except Exception as e:
    print("Sign-in modal did not appear, continuing...")

# Scroll down the page in a human-like fashion with a maximum scroll time
scroll_pause_time = 3  # Pause time between scrolls
max_scroll_time = 30  # Maximum scroll time in seconds
start_time = time.time()
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page
    time.sleep(scroll_pause_time)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height or (time.time() - start_time) > max_scroll_time:
        break
    last_height = new_height
time.sleep(1)
# Where to store the scraped data
items = []

# Select all property items on the page
property_items = driver.find_elements(By.CSS_SELECTOR, "[data-testid=\"property-card\"]")

print(f"Found {len(property_items)} property items.")


import csv

for property_item in property_items:
    # scraping logic...
    url = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "a[data-testid=\"property-card-desktop-single-image\"]").get_attribute("href"))
    image = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "img[data-testid=\"image\"]").get_attribute("src"))

    title = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "[data-testid=\"title\"]").text)
    address = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "[data-testid=\"address\"]").text)
    distance = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "[data-testid=\"distance\"]").text)

    review_score = None
    review_count = None
    review_text = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "[data-testid=\"review-score\"]").text)
    if review_text is not None:
    # split the review string by \n
        parts = review_text.split("\n")

    # process each part
    for part in parts:
        part = part.strip()
        # check if this part is a number (potential review score)
        if part.replace(".", "", 1).isdigit():
            review_score = float(part)
        # check if it contains the "reviews" string
        elif "reviews" in part:
            # extract the number before "reviews"
            review_count = int(part.split(" ")[0].replace(",", ""))

    decription = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "[data-testid=\"recommended-units\"]").text)

    price_element = handle_no_such_element_exception(lambda: (property_item.find_element(By.CSS_SELECTOR, "[data-testid=\"availability-rate-information\"]")))
    #if price_element is not None:
        #original_price = handle_no_such_element_exception(lambda: (
        #    price_element.find_element(By.CSS_SELECTOR, "[aria-hidden=\"true\"]:not([data-testid])").text.replace(",", "")
        #))
        #price = handle_no_such_element_exception(lambda: (
        #    price_element.find_element(By.CSS_SELECTOR, "[data-testid=\"price-and-discounted-price\"]").text.replace(",", "")
        #))

    # populate a new item with the scraped data
    item = {
    "url": url,
    "image": image,
    "title": title,
    "address": address,
    "distance": distance,
    "review_score": review_score,
    "review_count": review_count,
    "decription": decription
    #"original_price": original_price,
    #"price": price
    }
    # add the new item to the list of scraped items
    items.append(item)

output_file = "properties.csv"

with open(output_file, mode="w", newline="", encoding="utf-8") as file:
        #create a CSV writer object
    writer = csv.DictWriter(file, fieldnames=["url", "image", "title", "address", "distance", "review_score", "review_count", "decription", "original_price", "price"])

    # write the header row
    writer.writeheader()

    # write each item as a row in the CSV
    writer.writerows(items)
    driver.quit()


Sign-in modal did not appear, continuing...
Found 75 property items.


In [18]:
import re


hotel_names = []

for item in items:
    url = item['url']
    match = re.search(r'hotel/de/(.+?)\.de\.html', url)
    if match:
        hotel_name = match.group(1)
        hotel_names.append(hotel_name)

print(hotel_names)

['adlon-kempinski-berlin', 'nexliving-berlin-brandenburger-tor', 'marriott-am-potsdamer-platz', 'chateau-royal-gmbh-berlin62', 'hostel-near-the-brandenburg-gate', 'the-ritz-carlton-berlin', 'fairytale-in-sony-center-berlin', 'westingrand', 'the-mandala', 'maritimberlin', 'albrechtshof', 'hotelallegra', 'friedrichstrasse-apartments', 'jollyvivaldi', 'leonardo-berlin-mitte', 'eurostars-berlin', 'clipper-city-home', 'best-western-berlin-mitte', 'grand-hyatt-berlin', 'melia-berlin', 'gendarmenmarkt', 'the-mandala-berlin', 'nhberlinmitte', 'steigenberger-am-kanzleramt', 'artist-riverside-spa', 'meininger-berlin-hauptbahnhof', 'intercityhotel-berlin-hbf', 'gat-point-charlie', 'rocco-forte-de-rome', 'titanic-gendarmenmarkt-berlin', 'hilton-berlin', 'precise-tale-berlin', 'winter-s-gendarmenmarkt-berlin-mitte', 'wilde-aparthotels-by-staycity-berlin-checkpoint-charlie', 'winters-berlin-mitte-the-wall-checkpoint-charlie', 'dietrich-bonhoeffer', 'baxpax-downtown-hostel', 'ibis-berlin-hauptbahnhof

In [ ]:
for hotel_name in hotel_names:
    !python run.py {hotel_name} "de"


2024-12-02 21:29:07,831 - root - INFO - 

******** Input Params ********
-->  country: de
-->  hotel_name: adlon-kempinski-berlin
-->  sort_by: most_relevant
-->  n_rows: -1
-->  stop_critera: None
************************


2024-12-02 21:29:07,831 - root - INFO - Creating URLs
2024-12-02 21:29:07,832 - root - INFO - Checking max offset parameter value
2024-12-02 21:29:08,173 - root - INFO - Offset parameter max value: 4060
2024-12-02 21:29:08,196 - root - INFO - Created URLs: 407
2024-12-02 21:29:08,197 - root - INFO - Progress Monitoring Thread Started
2024-12-02 21:29:08,197 - root - INFO - Starting Get Requests on 407 urls
2024-12-02 21:29:48,654 - root - INFO - Finished Get Requests in 40.5 seconds
2024-12-02 21:29:48,654 - root - INFO - Starting Parsing Responses: 407
2024-12-02 21:29:50,210 - root - INFO - Processed 15/407
2024-12-02 21:29:52,211 - root - INFO - Processed 116/407
2024-12-02 21:29:52,517 - root - INFO - Processes launched: 5
2024-12-02 21:29:54,212 - root - INFO 

Process Process-3:
Traceback (most recent call last):
  File "c:\Users\alan_\anaconda3\envs\Scraping\Lib\multiprocessing\process.py", line 314, in _bootstrap
    self.run()
  File "c:\Users\alan_\anaconda3\envs\Scraping\Lib\multiprocessing\process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "c:\GitHub Projects\Hotel_review_project\Scraping\core\scrape.py", line 393, in _parse_scraped_results
    in review_text_liked.lower()
       ^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'lower'


2024-12-02 21:43:27,396 - root - INFO - 

******** Input Params ********
-->  country: de
-->  hotel_name: clipper-city-home
-->  sort_by: most_relevant
-->  n_rows: -1
-->  stop_critera: None
************************


2024-12-02 21:43:27,396 - root - INFO - Creating URLs
2024-12-02 21:43:27,396 - root - INFO - Checking max offset parameter value
2024-12-02 21:43:27,765 - root - INFO - Offset parameter max value: 1560
2024-12-02 21:43:27,774 - root - INFO - Created URLs: 157
2024-12-02 21:43:27,775 - root - INFO - Progress Monitoring Thread Started
2024-12-02 21:43:27,775 - root - INFO - Starting Get Requests on 157 urls
2024-12-02 21:43:43,148 - root - INFO - Finished Get Requests in 15.4 seconds
2024-12-02 21:43:43,148 - root - INFO - Starting Parsing Responses: 157
2024-12-02 21:43:45,780 - root - INFO - Processed 59/157
2024-12-02 21:43:46,966 - root - INFO - Processes launched: 5
2024-12-02 21:43:47,781 - root - INFO - Processed 147/157
2024-12-02 21:43:48,165 - root - INFO - Fin

In [27]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import random

def handle_no_such_element_exception(data_extraction_task):
    try:
        return data_extraction_task()
    except NoSuchElementException as e:
        return None

# Create a Chrome web driver instance
driver = webdriver.Chrome(service=Service())

# Connect to the target page
url = "https://www.booking.com/searchresults.de.html?label=gen173nr-1BCAEoggI46AdIM1gEaDuIAQGYAQe4ARfIAQzYAQHoAQGIAgGoAgO4Ap-wuLoGwAIB0gIkMWVhOGIyYzUtNzkwZC00ODI3LWE5Y2QtNzY3Mjg5YjA0NGI12AIF4AIB&sid=4daa01aeadda31e7f02b8ddc59e9f35f&aid=304142&ss=Berlin&ssne=Berlin&ssne_untouched=Berlin&efdco=1&lang=de&sb=1&src_elem=sb&dest_id=-1746443&dest_type=city&checkin=2025-05-05&checkout=2025-05-10&ltfd=5%3A1%3A6-2025%3A1%3A&group_adults=2&no_rooms=1&group_children=0&order=distance_from_search"
driver.get(url)

# Handle the sign-in alert
try:
    close_button = WebDriverWait(driver, 12).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[role=\"dialog\"] button[aria-label=\"Dismiss sign-in info.\"]"))
    )
    close_button.click()
except Exception as e:
    print("Sign-in modal did not appear, continuing...")

# Scroll down the page in a human-like fashion with a maximum scroll time
scroll_pause_time = 3  # Pause time between scrolls
max_scroll_time = 30  # Maximum scroll time in seconds
start_time = time.time()
last_height = driver.execute_script("return document.body.scrollHeight")
load_more_button_found = False
max_button_clicks = 4  # Maximum number of button clicks

while True:
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page
    time.sleep(scroll_pause_time)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height or (time.time() - start_time) > max_scroll_time:
        break
    last_height = new_height

    # Check if "Load More Results" button is present and click it
    if max_button_clicks > 0:
        try:
            load_more_button = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "button.a83ed08757.c21c56c305.bf0537ecb5.f671049264.af7297d90d.c0e0affd09")
                )
            )
            load_more_button.click()
            load_more_button_found = True
            print("Clicked 'Load More Results' button")
            # Add a small pause after clicking the button
            time.sleep(2)
            max_button_clicks -= 1
        except TimeoutException:
            pass  # Button not yet available, continue scrolling

# If "Load More Results" button was found, wait an additional time for final loading
if load_more_button_found:
    time.sleep(5)  # Adjust this wait time as needed

# Where to store the scraped data
items = []


# Select all property items on the page
property_items = driver.find_elements(By.CSS_SELECTOR, "[data-testid=\"property-card\"]")

print(f"Found {len(property_items)} property items.")


Sign-in modal did not appear, continuing...
Clicked 'Load More Results' button
Clicked 'Load More Results' button
Clicked 'Load More Results' button
Clicked 'Load More Results' button
Found 175 property items.


In [25]:
property_items

[<selenium.webdriver.remote.webelement.WebElement (session="5842cd1553a9f1ddc16326240b54ac1c", element="f.4558FE28F38630D5C6C8B1E559D0DDDC.d.DC40614A62DAC06EF315D712ACF5AC59.e.310")>,
 <selenium.webdriver.remote.webelement.WebElement (session="5842cd1553a9f1ddc16326240b54ac1c", element="f.4558FE28F38630D5C6C8B1E559D0DDDC.d.DC40614A62DAC06EF315D712ACF5AC59.e.311")>,
 <selenium.webdriver.remote.webelement.WebElement (session="5842cd1553a9f1ddc16326240b54ac1c", element="f.4558FE28F38630D5C6C8B1E559D0DDDC.d.DC40614A62DAC06EF315D712ACF5AC59.e.312")>,
 <selenium.webdriver.remote.webelement.WebElement (session="5842cd1553a9f1ddc16326240b54ac1c", element="f.4558FE28F38630D5C6C8B1E559D0DDDC.d.DC40614A62DAC06EF315D712ACF5AC59.e.313")>,
 <selenium.webdriver.remote.webelement.WebElement (session="5842cd1553a9f1ddc16326240b54ac1c", element="f.4558FE28F38630D5C6C8B1E559D0DDDC.d.DC40614A62DAC06EF315D712ACF5AC59.e.314")>,
 <selenium.webdriver.remote.webelement.WebElement (session="5842cd1553a9f1ddc163

In [28]:

import csv

for property_item in property_items:
    # scraping logic...
    url = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "a[data-testid=\"property-card-desktop-single-image\"]").get_attribute("href"))
    image = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "img[data-testid=\"image\"]").get_attribute("src"))
    time.sleep(random.uniform(1, 3))

    title = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "[data-testid=\"title\"]").text)
    address = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "[data-testid=\"address\"]").text)
    distance = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "[data-testid=\"distance\"]").text)
    time.sleep(random.uniform(1, 3))

    review_score = None
    review_count = None
    review_text = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "[data-testid=\"review-score\"]").text)
    if review_text is not None:
    # split the review string by \n
        parts = review_text.split("\n")

    # process each part
    for part in parts:
        part = part.strip()
        # check if this part is a number (potential review score)
        if part.replace(".", "", 1).isdigit():
            review_score = float(part)
        # check if it contains the "reviews" string
        elif "reviews" in part:
            # extract the number before "reviews"
            review_count = int(part.split(" ")[0].replace(",", ""))
    decription = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "[data-testid=\"recommended-units\"]").text)
    time.sleep(random.uniform(1, 3))
    price_element = handle_no_such_element_exception(lambda: (property_item.find_element(By.CSS_SELECTOR, "[data-testid=\"availability-rate-information\"]")))
    #if price_element is not None:
        #original_price = handle_no_such_element_exception(lambda: (
        #    price_element.find_element(By.CSS_SELECTOR, "[aria-hidden=\"true\"]:not([data-testid])").text.replace(",", "")
        #))
        #price = handle_no_such_element_exception(lambda: (
        #    price_element.find_element(By.CSS_SELECTOR, "[data-testid=\"price-and-discounted-price\"]").text.replace(",", "")
        #))

    # populate a new item with the scraped data
    item = {
    "url": url,
    "image": image,
    "title": title,
    "address": address,
    "distance": distance,
    "review_score": review_score,
    "review_count": review_count,
    "decription": decription
    #"original_price": original_price,
    #"price": price
    }
    # add the new item to the list of scraped items
    items.append(item)

output_file = "properties.csv"

with open(output_file, mode="w", newline="", encoding="utf-8") as file:
        #create a CSV writer object
    writer = csv.DictWriter(file, fieldnames=["url", "image", "title", "address", "distance", "review_score", "review_count", "decription", "original_price", "price"])

    # write the header row
    writer.writeheader()

    # write each item as a row in the CSV
    writer.writerows(items)
    driver.quit()
